In [0]:
df = spark.read.format("delta").table("workspace.bronze.crm_sales_details")

In [0]:
from pyspark.sql.functions import col, when, length, to_date

df = df.withColumn(
    "sls_order_dt",
    when(
        (col("sls_order_dt") == 0) | (length(col("sls_order_dt")) != 8),
        None
    ).otherwise(
        to_date(col("sls_order_dt").cast("string"), "yyyyMMdd")
    )
)

In [0]:
df = df.withColumn(
    "sls_ship_dt",
    when(
        (col("sls_ship_dt") == 0) | (length(col("sls_ship_dt")) != 8),
        None
    ).otherwise(
        to_date(col("sls_ship_dt").cast("string"), "yyyyMMdd")
    )
)

In [0]:
df = df.withColumn(
    "sls_due_dt",
    when(
        (col("sls_due_dt") == 0) | (length(col("sls_due_dt")) != 8),
        None
    ).otherwise(
        to_date(col("sls_due_dt").cast("string"), "yyyyMMdd")
    )
)

In [0]:
from pyspark.sql.functions import abs

df = df.withColumn(
    "sls_sales",
    when(
        (col("sls_sales").isNull()) |
        (col("sls_sales") <= 0) |
        (col("sls_sales") != col("sls_quantity") * abs(col("sls_price"))),
        col("sls_quantity") * abs(col("sls_price"))
    ).otherwise(col("sls_sales"))
)

In [0]:
from pyspark.sql.functions import when

df = df.withColumn(
    "sls_price",
    when(
        (col("sls_price").isNull()) | (col("sls_price") >= 0),
        col("sls_sales") / when(col("sls_quantity") == 0, None).otherwise(col("sls_quantity"))
    ).otherwise(col("sls_price"))
)

In [0]:
df.write.mode("overwrite").saveAsTable("silver.crm_sales_details")